# PyNEST Microcircuit: Run Simulation

This is an example script for running the microcircuit model and generating
basic plots of the network activity.


Import the necessary modules and start the time measurements.



In [2]:
import time

import nest
import network
import numpy as np
from network_params import net_dict
from sim_params import sim_dict
from stimulus_params import stim_dict

time_start = time.time()

XRT build version: 2.18.179
Build hash: 3ade2e671e5ab463400813fc2846c57edf82bb10
Build date: 2024-11-05 13:57:47
Git branch: 2024.2
PID: 1631782
UID: 1031
[Tue Aug 12 12:26:15 2025 GMT]
HOST: cyberdeck
EXE: /usr/bin/orted
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
XRT build version: 2.18.179
Build hash: 3ade2e671e5ab463400813fc2846c57edf82bb10
Build date: 2024-11-05 13:57:47
Git branch: 2024.2
PID: 1630443
UID: 1031
[Tue Aug 12 12:26:16 2025 GMT]
HOST


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.8.0
 Built: Jan 27 2025 08:18:03

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



/home/miahafiz/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning:Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
/home/miahafiz/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning:Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
/home/miahafiz/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning:Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.


Initialize the network with simulation, network and stimulation parameters,
then create and connect all nodes, and finally simulate.
The times for a presimulation and the main simulation are taken
independently. A presimulation is useful because the spike activity typically
exhibits a startup transient. In benchmark simulations, this transient should
be excluded from a time measurement of the state propagation phase. Besides,
statistical measures of the spike activity should only be computed after the
transient has passed.



In [2]:
net = network.Network(sim_dict, net_dict, stim_dict)
time_network = time.time()

net.create()
time_create = time.time()

net.connect()
time_connect = time.time()



Data will be written to: /home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/data/
  Directory already existed. Old data will be overwritten.

Neuron numbers are scaled by a factor of 0.001.
Indegrees are scaled by a factor of 0.001.
  Weights and DC input are adjusted to compensate.

RNG seed: 55
Total number of virtual processes: 1
Creating neuronal populations.

Aug 11 23:56:48 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.
Creating recording devices.
  Creating spike recorders.
Creating DC generators for external stimulation.
Connecting neuronal populations recurrently.
Connecting DC generators.

Aug 11 23:56:49 NodeManager::prepare_nodes [Info]: 
    Preparing 93 nodes for simulation.


/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/network.py:210: UserWarning:DC input created to compensate missing Poisson input.



In [3]:
print(net.num_neurons)
print(sum(net.num_neurons))

print(net.num_synapses)
print(sum(net.num_synapses))
print(sum(sum(net.num_synapses)))

print(net.ext_indegrees)
print(sum(net.ext_indegrees))

print(net.pops)
print(net.spike_recorders)
print(net.weight_matrix_mean)

[21  6 22  5  5  1 14  3]
77
[[45 22 20 10  3  0  2  0]
 [17  5  4  2  2  0  0  0]
 [ 4  1 24 17  1  0 15  0]
 [ 8  0 10  5  0  0  9  0]
 [11  2  6  0  2  2  1  0]
 [ 1  0  1  0  0  0  0  0]
 [ 5  1  7  1  4  0  8 11]
 [ 2  0  0  0  0  0  3  1]]
[93 31 72 35 12  2 38 12]
295
[2 2 2 2 2 2 3 2]
17
[NodeCollection(metadata=None, model=iaf_psc_exp, size=21, first=1, last=21), NodeCollection(metadata=None, model=iaf_psc_exp, size=6, first=22, last=27), NodeCollection(metadata=None, model=iaf_psc_exp, size=22, first=28, last=49), NodeCollection(metadata=None, model=iaf_psc_exp, size=5, first=50, last=54), NodeCollection(metadata=None, model=iaf_psc_exp, size=5, first=55, last=59), NodeCollection(metadata=None, model=iaf_psc_exp, size=1, first=60), NodeCollection(metadata=None, model=iaf_psc_exp, size=14, first=61, last=74), NodeCollection(metadata=None, model=iaf_psc_exp, size=3, first=75, last=77)]
NodeCollection(metadata=None, model=spike_recorder, size=8, first=78, last=85)
[[  2776.74837

In [43]:
def extract_syn_info_neuroring(net):
    """Extract synapse information from the network"""
    
    print("Extracting synapse information...")
    # create a list to store the synapse data
    synapse_data = []
    
    # First, extract connections between neuron populations (as before)
    for i, target_pop in enumerate(net.pops):
        for j, source_pop in enumerate(net.pops):
            if net.num_synapses[i][j] > 0:
                connections = nest.GetConnections(source=source_pop, target=target_pop)
                if len(connections) > 0:
                    conn_info = nest.GetStatus(connections, ['source', 'target', 'weight', 'delay'])
                    sources, targets, weights, delays = zip(*conn_info)
                    sources = np.array(sources)
                    targets = np.array(targets)
                    weights = np.array(weights)
                    delays = np.array(delays)
                    
                    # make the row source, target, delay, weight
                    # directly append to the synapse_data
                    synapse_data.append([sources, targets, delays, weights])
    
    # sort the row by source, so make similar source together
    synapse_data.sort(key=lambda x: x[0])
                    
    return synapse_data
                
                
def extract_synapse_info(net):
    """Extract detailed synapse information from the network including all node types"""
    
    print("Extracting synapse information...")
    synapse_data = {}
    
    # First, extract connections between neuron populations (as before)
    for i, target_pop in enumerate(net.pops):
        for j, source_pop in enumerate(net.pops):
            if net.num_synapses[i][j] > 0:
                print(f"Extracting connections from population {j} to population {i}")
                print(f"  Expected synapses: {net.num_synapses[i][j]}")
                
                connections = nest.GetConnections(source=source_pop, target=target_pop)
                
                if len(connections) > 0:
                    # GetStatus returns list of tuples: (source, target, weight, delay)
                    conn_info = nest.GetStatus(connections, ['source', 'target', 'weight', 'delay'])
                    sources, targets, weights, delays = zip(*conn_info)

                    # Convert to NumPy arrays
                    sources = np.array(sources)
                    targets = np.array(targets)
                    weights = np.array(weights)
                    delays = np.array(delays)

                    key = f"pop{j}_to_pop{i}"
                    synapse_data[key] = {
                        'source_pop': j,
                        'target_pop': i,
                        'source_type': 'neuron_population',
                        'target_type': 'neuron_population',
                        'source_neurons': sources,
                        'target_neurons': targets,
                        'weights': weights,
                        'delays': delays,
                        'num_synapses': len(sources)
                    }

                    print(f"  Actual synapses extracted: {len(sources)}")
                    print(f"  Weight range: {np.min(weights):.3f} to {np.max(weights):.3f} pA")
                    print(f"  Delay range: {np.min(delays):.3f} to {np.max(delays):.3f} ms\n")
                else:
                    print(f"  No connections found!\n")
        
    # Extract connections from neuron populations to spike_recorder
    print("Extracting connections from neuron populations to spike_recorder...")
    for i, source_pop in enumerate(net.pops):
        connections = nest.GetConnections(source=source_pop, target=net.spike_recorders)
        
        if len(connections) > 0:
            print(f"  Population {i} to spike_recorder")
            
            conn_info = nest.GetStatus(connections, ['source', 'target', 'weight', 'delay'])
            sources, targets, weights, delays = zip(*conn_info)

            sources = np.array(sources)
            targets = np.array(targets)
            weights = np.array(weights)
            delays = np.array(delays)

            key = f"pop{i}_to_spike_recorder"
            synapse_data[key] = {
                'source_pop': i,
                'target_pop': -2,  # -2 indicates spike_recorder
                'source_type': 'neuron_population',
                'target_type': 'spike_recorder',
                'source_neurons': sources,
                'target_neurons': targets,
                'weights': weights,
                'delays': delays,
                'num_synapses': len(sources)
            }

            print(f"    Synapses extracted: {len(sources)}")
            print(f"    Weight range: {np.min(weights):.3f} to {np.max(weights):.3f} pA")
            print(f"    Delay range: {np.min(delays):.3f} to {np.max(delays):.3f} ms\n")
    
    # Extract any other connections that might exist
    print("Extracting any other connections in the network...")
    
    # Get all connections in the network
    all_connections = nest.GetConnections()
    
    # Get all node IDs in the network
    all_nodes = []
    for pop in net.pops:
        all_nodes.extend(pop.tolist())
    all_nodes.extend(net.spike_recorders.tolist())
    all_nodes = set(all_nodes)
    
    # Find connections that haven't been captured yet
    captured_connections = set()
    for data in synapse_data.values():
        for i in range(len(data['source_neurons'])):
            captured_connections.add((data['source_neurons'][i], data['target_neurons'][i]))
        
    return synapse_data

def save_synapse_data(synapse_data, filename="synapse_data.txt"):
    """Save synapse data to a text file"""
    
    with open(filename, 'w') as f:
        f.write("Detailed Synapse Information\n")
        f.write("=" * 50 + "\n\n")
        
        for key, data in synapse_data.items():
            f.write(f"Connection: Population {data['source_pop']} → Population {data['target_pop']}\n")
            f.write(f"Number of synapses: {data['num_synapses']}\n")
            f.write(f"Weight range: {np.min(data['weights']):.3f} to {np.max(data['weights']):.3f} pA\n")
            f.write(f"Delay range: {np.min(data['delays']):.3f} to {np.max(data['delays']):.3f} ms\n")
            f.write("-" * 30 + "\n")
            
            # Write first 10 synapses as example
            f.write("First 10 synapses (source → target: weight, delay):\n")
            for k in range(min(10, len(data['source_neurons']))):
                f.write(f"  {data['source_neurons'][k]} → {data['target_neurons'][k]}: "
                       f"{data['weights'][k]:.3f} pA, {data['delays'][k]:.3f} ms\n")
            
            f.write("\n" + "=" * 50 + "\n\n")
    
    print(f"Synapse data saved to {filename}")

def save_synapse_csv(synapse_data, filename="synapses.csv"):
    """Save all synapses to a CSV file"""
    
    with open(filename, 'w') as f:
        f.write("source_neuron,target_neuron,weight,delay,source_pop,target_pop\n")
        
        for key, data in synapse_data.items():
            for i in range(len(data['source_neurons'])):
                f.write(f"{data['source_neurons'][i]},{data['target_neurons'][i]},"
                       f"{data['weights'][i]:.6f},{data['delays'][i]:.6f},"
                       f"{data['source_pop']},{data['target_pop']}\n")
    
    print(f"All synapses saved to {filename}")

def main():
    # Create and connect the network
    #print("Creating network...")
    #net = network.Network(sim_dict, net_dict, stim_dict)
    #net.create()
    #net.connect()
    
    # Extract synapse information
    synapse_data = extract_synapse_info(net)
    
    # Save data
    save_synapse_data(synapse_data)
    save_synapse_csv(synapse_data)
    
    # Print summary
    total_synapses = sum(data['num_synapses'] for data in synapse_data.values())
    print(f"\nTotal synapses extracted: {total_synapses}")
    
    # Show some statistics
    all_weights = np.concatenate([data['weights'] for data in synapse_data.values()])
    all_delays = np.concatenate([data['delays'] for data in synapse_data.values()])
    
    print(f"\nOverall statistics:")
    print(f"Weight mean: {np.mean(all_weights):.3f} pA, std: {np.std(all_weights):.3f} pA")
    print(f"Delay mean: {np.mean(all_delays):.3f} ms, std: {np.std(all_delays):.3f} ms")

if __name__ == "__main__":
    main() 

Extracting synapse information...
Extracting connections from population 0 to population 0
  Expected synapses: 45
  Actual synapses extracted: 45
  Weight range: 2098.304 to 3539.695 pA
  Delay range: 0.400 to 3.300 ms

Extracting connections from population 1 to population 0
  Expected synapses: 22
  Actual synapses extracted: 22
  Weight range: -12752.110 to -8681.262 pA
  Delay range: 0.100 to 1.400 ms

Extracting connections from population 2 to population 0
  Expected synapses: 20
  Actual synapses extracted: 20
  Weight range: 4153.749 to 6886.188 pA
  Delay range: 0.200 to 2.700 ms

Extracting connections from population 3 to population 0
  Expected synapses: 10
  Actual synapses extracted: 10
  Weight range: -12102.117 to -8945.970 pA
  Delay range: 0.300 to 0.900 ms

Extracting connections from population 4 to population 0
  Expected synapses: 3
  Actual synapses extracted: 3
  Weight range: 2517.834 to 2805.667 pA
  Delay range: 2.400 to 2.600 ms

Extracting connections from

In [ ]:
import struct
from collections import defaultdict

def extract_syn_info_neuroring(net):
    """Extract all synapse connections as a flat array: [source, target, delay, weight] per row."""
    print("Extracting synapse information...")
    synapse_list = []

    for i, target_pop in enumerate(net.pops):
        for j, source_pop in enumerate(net.pops):
            if net.num_synapses[i][j] > 0:
                connections = nest.GetConnections(source=source_pop, target=target_pop)
                if len(connections) > 0:
                    conn_info = nest.GetStatus(connections, ['source', 'target', 'weight', 'delay'])
                    for source, target, weight, delay in conn_info:
                        synapse_list.append([source, target, int(delay*10), weight])
    synapse_list.sort(key=lambda x: x[0])
    
    # create a new list to store array for neuroring kernel
    # --- Create packed_list ---
    if not synapse_list:
        return synapse_list, []

    # Find max source neuron
    max_source = max(row[0] for row in synapse_list)
    block_size = 5000
    packed_list = [0] * ((max_source + 1) * block_size)

    # Group synapses by source
    source_dict = defaultdict(list)
    for row in synapse_list:
        source_dict[row[0]].append(row)

    for source, syns in source_dict.items():
        base_idx = source * block_size
        packed_list[base_idx] = len(syns)
        print(f"Filling block for source {source} at index {base_idx}, count {len(syns)}")
        for i, (src, tgt, dly, wgt) in enumerate(syns):
            # target: 24 bits, delay: 8 bits, weight: 32 bits float
            tgt = int(tgt) & 0xFFFFFF
            dly = int(dly) & 0xFF
            # pack weight as float32 bits
            wgt_bits = struct.unpack('>I', struct.pack('>f', float(wgt)))[0]  # big-endian
            packed = (wgt_bits << 32) | (dly << 24) | tgt
            packed_list[base_idx + 1 + i] = packed

    return synapse_list, packed_list

synapse_list, packed_list = extract_syn_info_neuroring(net)
print(synapse_list)
print(packed_list[1*5000 : 1*5000+10])
print(packed_list[2*5000 : 2*5000+10])
print(packed_list[3*5000 : 3*5000+10])
print(packed_list[4*5000 : 4*5000+10])
print(packed_list[5*5000 : 5*5000+10])
print(packed_list[6*5000 : 6*5000+10])
print(packed_list[7*5000 : 7*5000+10])
print(packed_list[8*5000 : 8*5000+10])
print(packed_list[9*5000 : 9*5000+10])
print(packed_list[10*5000 : 10*5000+10])

Extracting synapse information...


NESTErrors.DictError: DictError in SLI function get_d: Key '/source_pop' does not exist in dictionary.

In [4]:
print(packed_list[1*5000 : 1*5000+10])

NameError: name 'packed_list' is not defined

In [5]:
    synapse_list = []

    for i, target_pop in enumerate(net.pops):
        for j, source_pop in enumerate(net.pops):
            if net.num_synapses[i][j] > 0:
                connections = nest.GetConnections(source=source_pop, target=target_pop)
                if len(connections) > 0:
                    conn_info = nest.GetStatus(connections, ['source', 'target', 'weight', 'delay'])
                    for source, target, weight, delay in conn_info:
                        synapse_list.append([source, target, int(delay*10), weight])
    synapse_list.sort(key=lambda x: x[0])
    
    connect_info = nest.GetConnections(source=net.dc_stim_input, target=net.pops[0])

    print(net.DC_amp)

[ 58.89137578 175.80370122 183.31554282 183.95581312 199.83483291
 235.92304062  94.30746371 241.01265922]


In [4]:
dc_amp_bits = net_dict["K_ext"]*0.3
print(dc_amp_bits)

[480. 450. 630. 570. 600. 570. 870. 630.]


In [ ]:
all_connections = nest.GetConnections()

print(nest.GetConnections())


# get network info
print(net.num_neurons)
print(sum(net.num_neurons))
print(net.num_synapses)
print(sum(net.num_synapses))
print(net.ext_indegrees)
print(sum(net.ext_indegrees))
print(net.pops)
print(net.spike_recorders)
print(net.dc_stim_input)

connect_info = nest.GetConnections(source=net.dc_stim_input, target=net.pops[0])
print(net.dc_stim_input.origin)
print(net.dc_stim_input.stop)
print(net.dc_stim_input.stimulus_source)
print(net.dc_stim_input.start)
print(net.dc_stim_input.amplitude)


# get list of neuron in population 4
print()

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1       20  static_synapse    2679.   1.600
      1       63  static_synapse    2653.   2.000
      1        8  static_synapse    2706.  0.9000
      1        5  static_synapse    2925.   1.900
      1       59  static_synapse    2560.   1.400
      1        3  static_synapse    2859.   1.200
      2       23  static_synapse    3039.   1.400
      2        6  static_synapse    2810.  0.5000
      3       62  static_synapse    2256.   2.000
      3       15  static_synapse    2953.   1.500
      3       56  static_synapse    2830.   2.500
      5        1  static_synapse    2599.   2.100
      5       55  static_synapse    2898.  0.7000
      5       10  static_synapse    2399.   1.000
      6       51  static_synapse    3037.   1.800
     ⋮        ⋮               ⋮        ⋮       ⋮ 
     92       63  static_synapse    1.000   1.000
     92       64  static_synapse    1.000   1.00

In [19]:
# get v0 for all neurons in population 1
v0_values = net.pops[1].get('V_m')
print(net.pops[0].get('V_m'))

(-59.398244404166775, -65.42416946213079, -66.32029899483236, -61.767508416554456, -61.89270549660911, -68.74338588040239, -65.83176702993345, -73.86056500024401, -70.90953993839035, -66.57801291750745, -74.04612796834354, -66.99364011457321, -64.12123729441599, -63.051890840638116, -69.80003051776427, -72.82221927903234, -61.92992608874837, -70.89608219432839, -73.87026953710645, -70.9953428541527, -63.67150013091096)


In [17]:
k = 0  # population index
i_e_values = net.pops[k].get('I_e')  # array, same value for all neurons in pop k
print(i_e_values)

print(net.DC_amp)

print(net.pops[1][0].global_id)

(58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234, 58.89137577624234)
[ 58.89137578 175.80370122 183.31554282 183.95581312 199.83483291
 235.92304062  94.30746371 241.01265922]
22


In [18]:
synapse_list = []
neuron_info = []
for i, source_pop in enumerate(net.pops):
    for j, target_pop in enumerate(net.pops):
        if net.num_synapses[i][j] > 0:
            connections = nest.GetConnections(source=source_pop, target=target_pop)
            if len(connections) > 0:
                conn_info = nest.GetStatus(connections, ['source', 'target', 'weight', 'delay'])
                for source, target, weight, delay in conn_info:
                    synapse_list.append([source, target, int(delay*10), weight, i, net.DC_amp[i], net.pops[i].get('V_m')[source - net.pops[i][0].global_id]])
            # sort synapse_list by source
        synapse_list.sort(key=lambda x: x[0])
        print(synapse_list)
        # calculate how many synapses for each source neuron
        print("-------------------------------------")

    

print(synapse_list)
print(neuron_info)

[[1, 20, 16, 2678.9072171000325, 0, 58.89137577624234, -59.398244404166775], [1, 8, 9, 2706.346438190091, 0, 58.89137577624234, -59.398244404166775], [1, 5, 19, 2924.8885460340184, 0, 58.89137577624234, -59.398244404166775], [1, 3, 12, 2858.966631551489, 0, 58.89137577624234, -59.398244404166775], [2, 6, 5, 2809.5987661395884, 0, 58.89137577624234, -65.42416946213079], [3, 15, 15, 2953.175237316281, 0, 58.89137577624234, -66.32029899483236], [5, 1, 21, 2598.829055592843, 0, 58.89137577624234, -61.89270549660911], [5, 10, 10, 2398.764775239432, 0, 58.89137577624234, -61.89270549660911], [6, 14, 13, 3446.050699159521, 0, 58.89137577624234, -68.74338588040239], [6, 11, 14, 2819.572519722036, 0, 58.89137577624234, -68.74338588040239], [6, 3, 9, 2771.6996374316927, 0, 58.89137577624234, -68.74338588040239], [6, 14, 26, 2552.423212752896, 0, 58.89137577624234, -68.74338588040239], [6, 11, 13, 3066.825860428111, 0, 58.89137577624234, -68.74338588040239], [7, 18, 20, 2955.1616567997075, 0, 58.

In [20]:
        num_compute_units = 4
        num_fpgas = 1
        devices = []
        xclbins = []
        uuids = []
        kernels = []  # List of NeuroRingKernel instances
        kernels_per_fpga = []  # List of lists: kernels assigned to each FPGA
        spikeRecorder_array = []
        # --- Extract synapse_list and packed_list ---
        print("Extracting synapse information...")
        synapse_list = []
        for i, source_pop in enumerate(net.pops):
            for j, target_pop in enumerate(net.pops):
                if net.num_synapses[i][j] > 0:
                    connections = nest.GetConnections(source=source_pop, target=target_pop)
                    if len(connections) > 0:
                        conn_info = nest.GetStatus(connections, ['source', 'target', 'weight', 'delay'])
                        for source, target, weight, delay in conn_info:
                            synapse_list.append([source, target, int(delay*10), weight, i, net.DC_amp[i], net.pops[i].get('V_m')[source - net.pops[i][0].global_id]])
        synapse_list.sort(key=lambda x: x[0])
        
        # 1. Group synapses by source neuron
        source_dict = defaultdict(list)
        for row in synapse_list:
            source_dict[row[0]].append(row)
        # 2. Get all unique source neurons, sorted for reproducibility
        sources = sorted(source_dict.keys())

        # 3. Calculate CUs per FPGA
        num_cus_per_fpga = [num_compute_units // num_fpgas + (1 if i < num_compute_units % num_fpgas else 0) for i in range(num_fpgas)]

        # 4. Split sources into groups for each CU, grouped by FPGA
        synapse_list_per_fpga = []
        cu_idx = 0
        for fpga_idx, cu_count in enumerate(num_cus_per_fpga):
            fpga_cu_synapses = []
            for cu in range(cu_count):
                # Calculate the range of sources for this CU
                start = cu_idx * len(sources) // num_compute_units
                end = (cu_idx + 1) * len(sources) // num_compute_units
                group_sources = sources[start:end]
                group_synapses = []
                for src in group_sources:
                    group_synapses.extend(source_dict[src])
                fpga_cu_synapses.append(group_synapses)
                cu_idx += 1
            synapse_list_per_fpga.append(fpga_cu_synapses)
            
        # --- Create packed_list ---
        packed_list_per_fpga = []
        kernel_neuron_ranges_per_fpga = []
        block_size = 10000

        for fpga_cu_synapses in synapse_list_per_fpga:
            fpga_packed_lists = []
            fpga_neuron_ranges = []
            for cu_synapses in fpga_cu_synapses:
                cu_source_dict = defaultdict(list)
                for row in cu_synapses:
                    cu_source_dict[row[0]].append(row)
                if cu_source_dict:
                    min_src = min(cu_source_dict.keys())
                    max_src = max(cu_source_dict.keys())
                    neuron_start = min_src
                    neuron_total = max_src - min_src + 1
                else:
                    neuron_start = -1
                    neuron_total = 0
                fpga_neuron_ranges.append((neuron_start, neuron_total))
                cu_packed_list = [0] * (neuron_total * block_size)
                if neuron_total > 0 and neuron_start >= 0:
                    for idx, src in enumerate(range(neuron_start, neuron_start + neuron_total)):
                        syns = cu_source_dict.get(src, [])
                        block_start = idx * block_size
                        if syns:
                            cu_packed_list[block_start] = len(syns)
                            # Store DC amplitude as IEEE-754 float bits (big-endian), consistent with weights
                            dc_amp_bits = struct.unpack('>I', struct.pack('>f', float(np.average(net.DC_amp))))[0]
                            cu_packed_list[block_start + 1] = dc_amp_bits
                            print(syns)
                            for i, (src, tgt, dly, wgt) in enumerate(syns):
                                tgt = int(tgt) & 0xFFFFFF
                                dly = int(dly) & 0xFF
                                packed_td = (tgt << 8) | dly  # target_id upper 24 bits, delay lower 8 bits
                                wgt_bits = struct.unpack('>I', struct.pack('>f', float(wgt)))[0]
                                cu_packed_list[block_start + 16 + 2*i] = packed_td
                                cu_packed_list[block_start + 17 + 2*i] = wgt_bits
                        # else: block is already zeroed
                cu_packed_list = np.array(cu_packed_list, dtype=np.uint32)
                fpga_packed_lists.append(cu_packed_list)
            packed_list_per_fpga.append(fpga_packed_lists)
            kernel_neuron_ranges_per_fpga.append(fpga_neuron_ranges)


Extracting synapse information...


NameError: name 'defaultdict' is not defined

In [17]:
net.simulate(sim_dict["t_presim"])
time_presimulate = time.time()

net.simulate(sim_dict["t_sim"])
time_simulate = time.time()

Simulating 500.0 ms.
Simulating 1000.0 ms.


Plot a spike raster of the simulated neurons and a box plot of the firing
rates for each population.
For visual purposes only, spikes 100 ms before and 100 ms after the thalamic
stimulus time are plotted here by default.
The computation of spike rates discards the presimulation time to exclude
initialization artifacts.



In [18]:
raster_plot_interval = np.array([stim_dict["th_start"] - 100.0, stim_dict["th_start"] + 100.0])
firing_rates_interval = np.array([sim_dict["t_presim"], sim_dict["t_presim"] + sim_dict["t_sim"]])
net.evaluate(raster_plot_interval, firing_rates_interval)
time_evaluate = time.time()

Interval to plot spikes: [600. 800.] ms


/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/helpers.py:426: UserWarning:loadtxt: input contained no data: "/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/data/spike_recorder-78-0.dat"
/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/helpers.py:426: UserWarning:loadtxt: input contained no data: "/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/data/spike_recorder-79-0.dat"
/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/helpers.py:426: UserWarning:loadtxt: input contained no data: "/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/data/spike_recorder-80-0.dat"
/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/helpers.py:426: UserWarning:loadtxt: input contained no data: "/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/data/spike_recorder-81-0.dat"
/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/helpers.py:426: UserWarning:loadtxt: input contained no data: "/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/data/spike_recorder-82-0.

Interval to compute firing rates: [ 500. 1500.] ms
Mean rates: [0. 0. 0. 0. 0. 0. 0. 0.] spikes/s
Standard deviation of rates: [0. 0. 0. 0. 0. 0. 0. 0.] spikes/s


/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/helpers.py:426: UserWarning:loadtxt: input contained no data: "/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/data/spike_recorder-78-0.dat"
/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/helpers.py:426: UserWarning:loadtxt: input contained no data: "/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/data/spike_recorder-79-0.dat"
/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/helpers.py:426: UserWarning:loadtxt: input contained no data: "/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/data/spike_recorder-80-0.dat"
/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/helpers.py:426: UserWarning:loadtxt: input contained no data: "/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/data/spike_recorder-81-0.dat"
/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/helpers.py:426: UserWarning:loadtxt: input contained no data: "/home/miahafiz/NeuroRing/host_py/corticalmicrocircuit/data/spike_recorder-82-0.

Summarize time measurements. Rank 0 usually takes longest because of the
data evaluation and print calls.



In [16]:
print(
    "\nTimes of Rank {}:\n".format(nest.Rank())
    + "  Total time:          {:.3f} s\n".format(time_evaluate - time_start)
    + "  Time to initialize:  {:.3f} s\n".format(time_network - time_start)
    + "  Time to create:      {:.3f} s\n".format(time_create - time_network)
    + "  Time to connect:     {:.3f} s\n".format(time_connect - time_create)
    + "  Time to presimulate: {:.3f} s\n".format(time_presimulate - time_connect)
    + "  Time to simulate:    {:.3f} s\n".format(time_simulate - time_presimulate)
    + "  Time to evaluate:    {:.3f} s\n".format(time_evaluate - time_simulate)
)

NameError: name 'time_presimulate' is not defined